In [5]:
#READ DATA
using CSV, DataFrames, JuMP, Gurobi

df_nut = CSV.read("data_nutrients.csv", DataFrame, delim = ";")
df_rec = CSV.read("data_recipes.csv", DataFrame, delim = ";")
df_price = CSV.read("price_ing.csv", DataFrame, delim = ";")
df_recConst = CSV.read("recipes_constr_cleaned.csv", DataFrame)
;

In [6]:
# Create dictionary that maps recipe names to lists of ingredients
mydict = Dict(
    Pair.(
        transform(combine(groupby(df_recConst, :Name), :Match => Set)).Name,
        transform(combine(groupby(df_recConst, :Name), :Match => Set)).Match_Set
    )
)
;

In [4]:
# Create dictionary that maps meal number (1 = breakfast, 2 = lunch, 3 = dinner) to possible recipes
stacked = subset(
    stack(
        unique(select(df_recConst, :Name, :B, :L, :D, :S)), 
        2:5
    ),
    :value => ByRow(value -> value),
    skipmissing=true
)

temp_dict = Dict(
    Pair.(
        transform(combine(groupby(stacked, :variable), :Name => Set)).variable,
        transform(combine(groupby(stacked, :variable), :Name => Set)).Name_Set
    )
)

mydict2 = Dict(
    1 => temp_dict["B"],
    2 => temp_dict["L"],
    3 => temp_dict["D"],
    # 4 = > temp_dict["S"] # Uncomment to include snack/dessert items
)
;

In [15]:
#DEFINE SETS AND INDICES

#----------------------------------------------------------------------
#set of days (d)
d = 5
D = [j for j in range(1,d)]

#set of meals (m)
m = 3
M = [j for j in range(1,m)]

#set of nutrients (n)
N = unique(df_nut, "nutrients").nutrients

#set of nutrients relaxed daily lower bound
lownut = copy(N)
deleteat!(lownut, findall(x->x=="Calories",lownut))
deleteat!(lownut, findall(x->x=="Fat",lownut))
deleteat!(lownut, findall(x->x=="Protein",lownut))
deleteat!(lownut, findall(x->x=="Carb",lownut))

#set of nutrients with daily upper bound
uppnut = ["Calories"]

#set of nutrients with per meal lower and upper bound
uppnutmeal = ["Calories"]

#set of recipes (r)
R = unique(df_recConst[:,1])

#set of people (p)
P = unique(df_nut, "person").person

#set of ingredients (i)
I = unique(df_recConst, "Match").Match

#----------------------------------------------------------------------
#indice rmd
rmd = [(l,j,k) for l in R for j in M for k in D if l in mydict2[j]]

#indice rmdp
rmdp = [(l,j,k,f) for l in R for j in M for k in D for f in P if l in mydict2[j]]

#indice np
np = [(u,f) for u in N for f in P]

#indice nr
nr =[(u,l) for u in N for l in R]

#indice ir
ir = [(row.Match,row.Name) for row in eachrow(df_recConst)]
;

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1206 rows, 2015 columns and 64091 nonzeros
Model fingerprint: 0x5dad5525
Variable types: 1528 continuous, 487 integer (468 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+03]
  Objective range  [9e-04, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+03]
Presolve removed 124 rows and 490 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 10 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-callback calls 45, time in user-callback 0.00 sec


In [16]:
#DEFINE DATA

#daily needs of nutrient n by person p
need = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :lowerbound][1] for (j,k) in np)

#daily max of nutrient n by person p
####max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for (j,k) in np)
max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for j in uppnut for k in P)

#price of ingredient i per unit of measurement of the ingredient
price = Dict(j => df_price[(df_price.ingredient .== j), :price][1] for j in I)

#quantity of nutrient n granted by one portion of recipe r
nutrec = Dict((j,k) => df_rec[(df_rec.recipe .== k), j][1] for (j,k) in nr)

#quantity of ingredient i needed by one portion of recipe r
recing = Dict((j,k) => df_recConst[(df_recConst.Name .== k) .& (df_recConst.Match .== j), :SI_Qty_Per_Serving][1]
    for (j,k) in ir)

#quantity of calories per unit of macros
qtycal = Dict("Protein" => 4, "Carb" => 4, "Fat" => 9)

#minimum contribution from each macro to calories
ratio = Dict("Protein" => 0.10, "Carb" => 0.45, "Fat" => 0.20)

#maximum contribution from each macro to calories
ratio2 = Dict("Protein" => 0.35, "Carb" => 0.65, "Fat" => 0.35)

#minimum percentage of calories in each meal
perc = [0.10,0.35,0.30]

#maximum percentage of calories in each meal
perc2 = [0.25,0.55,0.50]

#maximum percentage of meals for which the same recipe can be chosen
samerecperc = 0.1

#for (j,k) in nr
    #println(j,", ",k,": ",nutrec[(j,k)])
#end
;

In [17]:
#DEFINE AND SHOW MODEL

#--model
mdl = Model(with_optimizer(Gurobi.Optimizer))

#--decision variables
@variables mdl begin
    X[rmd], Bin
    Q[rmdp]>=0
    Y[I]>=0 #need to understand if it is possible to define some of these as integer and others as continuous
end

#change variable Y to be integer for certain ingredients
for s in I
    if df_price[(df_price.ingredient .== s), :unit][1] == "each" || df_price[(df_price.ingredient .== s), :unit][1] == "ct"
        set_integer(Y[s])
    end
end

#--objective function
@objective(mdl, Min, 0.99999*sum(Y[j]*price[j] for j in I)
    - 0.00001*sum((sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) - need[(j,l)]) for i in D, 
            j in lownut, l in P))

#--constraints
@constraints mdl begin
    constraint_1[i in M,j in D],
    sum(X[(l,v,s)] for (l,v,s) in rmd if v==i && s==j) == 1
    
    constraint_2[i in D,j in ["Calories"],l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) >= need[(j,l)] 

    constraint_3[(i,j,l) in rmd],
    sum(Q[(i,j,l,k)] for k in P) <= 1000*X[(i,j,l)]
    
    constraint_4[s in I],
    sum(Q[(i,j,l,k)]*recing[(s,i)] for (i,j,l,k) in rmdp if s in mydict[i]) <= Y[s]
    
    constraint_5[i in D,j in uppnut,l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) <= max[(j,l)]
    
    constraint_6[k in M,i in D,j in uppnutmeal,l in P],
    (sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l && v==k)
       >= perc[k]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l))
    
    constraint_7[k in M,i in D,j in uppnutmeal,l in P],
    (sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l && v==k)
       <= perc2[k]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l))
    
    constraint_8[i in mydict2[2]],
    sum(X[(h,v,s)] for (h,v,s) in rmd if h==i) - samerecperc*(d*(m-1)) <= 0 #(v in [2,3] &&)
    
    constraint_9[i in mydict2[2], j in D],
    sum(X[(h,v,s)] for (h,v,s) in rmd if h==i && s==j) <= 1
    
    constraint_10[i in ["Protein","Carb","Fat"],j in ["Calories"], k in D, l in P],
    sum(Q[(h,v,s,a)]*nutrec[(i,h)]*qtycal[i] for (h,v,s,a) in rmdp if s==k && a==l) >= ratio[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l)
    
    constraint_11[i in ["Protein","Carb","Fat"],j in ["Calories"], k in D, l in P],
    sum(Q[(h,v,s,a)]*nutrec[(i,h)]*qtycal[i] for (h,v,s,a) in rmdp if s==k && a==l) <= ratio2[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l)
    
    constraint_12[i in D,j in lownut,l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) >= 0.1
end
;

Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-16


In [19]:
#--solve the model
optimize!(mdl)

choosen_recipes = value.(X)
quantities = value.(Q)
needed_ing = value.(Y)

result = objective_value(mdl)

;

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1874 rows, 3263 columns and 106723 nonzeros
Model fingerprint: 0x7661f518
Variable types: 2464 continuous, 799 integer (780 binary)
Coefficient statistics:
  Matrix range     [3e-02, 2e+03]
  Objective range  [9e-04, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 2e+03]
Presolved: 1414 rows, 3198 columns, 71692 nonzeros

Continuing optimization...


Cutting planes:
  Gomory: 2
  Cover: 30
  Implied bound: 555
  MIR: 379
  Flow cover: 391
  GUB cover: 2
  Relax-and-lift: 100

Explored 9016 nodes (590409 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 10 (of 10 available processors)

Solution count 10: 33.4059 33.4111 33.8059 ... 48.364

Optimal solution found (tolerance 1.00e-04)
Best objective 3.340592316103e+01, best bound 3.340592316103e+01, gap 0.0000%

User-callback calls 21141, t

In [257]:
# Print table of recipes with serving sizes for each person
df = DataFrame(mapreduce(permutedims, vcat, [[x, y, z, k] for (x, y, z, k) in quantities.axes[1][quantities.data .> 0]]), :auto)
df.Servings = quantities.data[quantities.data .>0]
df = rename(df, :x1 => :Recipe, :x2 => :Meal, :x3 => :Day, :x4 => :Person)
show(df)
CSV.write("menu.csv", df);

45×5 DataFrame
 Row │ Recipe                             Meal  Day  Person  Servings 
     │ Any                                Any   Any  Any     Float64  
─────┼────────────────────────────────────────────────────────────────
   1 │ Apple-Stuffed Squash               1     3    1        3.53846
   2 │ Apple-Stuffed Squash               1     3    2        3.60577
   3 │ Apple-Stuffed Squash               1     3    3        3.17308
   4 │ Baked Pork Chops                   2     3    1        3.47895
   5 │ Baked Pork Chops                   2     3    2        3.54512
   6 │ Baked Pork Chops                   2     3    3        3.11971
   7 │ Banana Cupcakes                    1     1    1        2.18009
   8 │ Banana Cupcakes                    1     1    2        2.22156
  ⋮  │                 ⋮                   ⋮     ⋮     ⋮        ⋮
  39 │ Roast Chicken                      2     4    3        2.06787
  40 │ Spinach and Meat Cakes             3     1    1        1.39899
  41 │

In [258]:
# Print nutritional content from each meal for each person
df2 = rightjoin(df_rec, df, on = :recipe => :Recipe)
df2 = select(df2, Not([:Meal, :Day, :Person, :Servings]))
df2 = df2[:,2:end].*df.Servings
df2.Recipe = df.Recipe
df2 = leftjoin(df, df2, on = :Recipe => :Recipe)
show(df2)
CSV.write("results.csv", df2);


153×33 DataFrame
 Row │ Recipe                  Meal  Day  Person  Servings  Calories  Fat      ⋯
     │ Any                     Any   Any  Any     Float64   Float64?  Float64? ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Apple-Stuffed Squash    1     3    1        3.53846   460.0     7.07692 ⋯
   2 │ Apple-Stuffed Squash    1     3    2        3.60577   460.0     7.07692
   3 │ Apple-Stuffed Squash    1     3    3        3.17308   460.0     7.07692
   4 │ Apple-Stuffed Squash    1     3    1        3.53846   468.75    7.21154
   5 │ Apple-Stuffed Squash    1     3    2        3.60577   468.75    7.21154 ⋯
   6 │ Apple-Stuffed Squash    1     3    3        3.17308   468.75    7.21154
   7 │ Apple-Stuffed Squash    1     3    1        3.53846   412.5     6.34615
   8 │ Apple-Stuffed Squash    1     3    2        3.60577   412.5     6.34615
  ⋮  │           ⋮              ⋮     ⋮     ⋮        ⋮         ⋮         ⋮     ⋱
 147 │ Vegetable Chili 

In [285]:
df3 = DataFrame(Ingredient=needed_ing.axes[1][needed_ing.data .> 0], Qty=needed_ing.data[needed_ing.data .> 0])
prices = DataFrame(Ingredient=collect(keys(price)), Price=collect(values(price)))
df3 = leftjoin(df3, prices, on = :Ingredient)
df3.Subtotal = df3.Qty .* df3.Price
show(df3)
CSV.write("shopping_list.csv", df3);

55×4 DataFrame
 Row │ Ingredient                         Qty         Price        Subtotal   
     │ String                             Float64     Float64?     Float64    
─────┼────────────────────────────────────────────────────────────────────────
   1 │ Yellow Bell Peppers                  0.633538  0.00659182   0.00417617
   2 │ Spur Tree Curry Seasoning           18.7359    0.0116051    0.217433
   3 │ Red Bell Peppers                   199.991     0.0043872    0.8774
   4 │ Jalapeno Peppers                     1.77333   0.0043872    0.00777996
   5 │ Ground Beef 80 Lean                702.222     0.00945783   6.6415
   6 │ Lemons                               1.0       0.39         0.39
   7 │ Badia Cinnamon Powder               16.1766    0.0486781    0.787448
   8 │ Badia Ground Cumin                  21.9117    0.024339     0.533311
  ⋮  │                 ⋮                      ⋮            ⋮           ⋮
  49 │ Baby Spinach                       291.154     0.0043872    1.27